In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer  

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
frame = pd.read_csv('20-200.csv')
print('数据量：%d'%frame.shape[0],'类目数：%d'%len(Counter(frame['标签'])))

数据量：46399 类目数：768


In [3]:
from tensorflow.contrib import learn
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length = 15,min_frequency = 10)
x_text = np.array(list(vocab_processor.fit_transform(frame['商品描述'])))

In [4]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame['标签'])

In [6]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random

input_size = 100
timestep_size = 15 
hidden_size = 256
layer_num = 2
class_num = len(Counter(frame['标签']))
learning_rate = 0.001
vocabulary_size = len(vocab_processor.vocabulary_)
embed_dim = 100
sequence_length = 15

with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    y = tf.placeholder(tf.int64,[None])
    input_data = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_data')  

with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    #lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
with tf.name_scope("embedding_layer"):
    embedding = tf.get_variable("embedding",[vocabulary_size,embed_dim],dtype=tf.float32)
    inputs=tf.nn.embedding_lookup(embedding,input_data)    
#input_data (batch_size , sequence_length)
#inputs shape : (batch_size , sequence_length , embed_dim)

inputs = tf.transpose(inputs, [1,0,2])
#inputs shape : (sequence_length, batch_size, embed_dim)

inputs = tf.reshape(inputs, [-1, embed_dim])

# 转换成list,里面的每个元素是(batch_size, embed_dim)
inputs = tf.split(inputs, sequence_length, 0) 
with tf.name_scope("output"):
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, inputs, dtype=tf.float32)

In [7]:
attention_size = 256
with tf.name_scope('attention'), tf.variable_scope('attention'):
    attention_w = tf.Variable(tf.truncated_normal([2*hidden_size, attention_size], stddev=0.1), name='attention_w')
    attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
    u_list = []
    for t in range(15):
        u_t = tf.tanh(tf.matmul(outputs[t], attention_w) + attention_b) 
        u_list.append(u_t)
    u_w = tf.Variable(tf.truncated_normal([attention_size, 1], stddev=0.1), name='attention_uw')
    attn_z = []
    for t in range(15):
        z_t = tf.matmul(u_list[t], u_w)
        attn_z.append(z_t)
    # transform to batch_size * sequence_length
    attn_zconcat = tf.concat(attn_z, axis=1)
    alpha = tf.nn.softmax(attn_zconcat)
    # transform to sequence_length * batch_size * 1 , same rank as outputs
    alpha_trans = tf.reshape(tf.transpose(alpha, [1,0]), [15, -1, 1])
    final_output = tf.reduce_sum(outputs * alpha_trans, 0)
with tf.name_scope("result"):
    fc_w = tf.Variable(tf.truncated_normal([2*hidden_size, class_num], stddev=0.1), name='fc_w')
    fc_b = tf.Variable(tf.zeros([class_num]), name='fc_b')
    #self.final_output = outputs[-1]
    # 用于分类任务, outputs取最终一个时刻的输出
    logits = tf.matmul(final_output, fc_w) + fc_b
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(x_text,y_label,test_size = 0.1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(5001):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={input_data:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={input_data:temp_x,y:temp_y,keep_prob: 0.4, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.0012931
Iter 3: step 101, training accuracy 0.0948276
Iter 5: step 201, training accuracy 0.3125
Iter 8: step 301, training accuracy 0.497629
Iter 10: step 401, training accuracy 0.622629
Iter 13: step 501, training accuracy 0.67306
Iter 15: step 601, training accuracy 0.703664
Iter 18: step 701, training accuracy 0.721983
Iter 20: step 801, training accuracy 0.724138
Iter 22: step 901, training accuracy 0.733405
Iter 25: step 1001, training accuracy 0.734698
Iter 27: step 1101, training accuracy 0.745905
Iter 30: step 1201, training accuracy 0.737931
Iter 32: step 1301, training accuracy 0.734483
Iter 35: step 1401, training accuracy 0.736638
Iter 37: step 1501, training accuracy 0.742457
Iter 40: step 1601, training accuracy 0.74569
Iter 42: step 1701, training accuracy 0.744612
Iter 44: step 1801, training accuracy 0.750647
Iter 47: step 1901, training accuracy 0.749784
Iter 49: step 2001, training accuracy 0.745043
Iter 52: step 2101, training ac